In [12]:
import numpy as np
import pandas as pd

def bit_length(arr):
    return np.frexp(arr)[1]

empty = [-1, -1]

In [13]:
SUBS = {
    "0": "₀",
    "1": "₁",
    "2": "₂",
    "3": "₃",
    "4": "₄",
    "5": "₅",
    "6": "₆",
    "7": "₇",
    "8": "₈",
    "9": "₉",
}


def subscript(num):
    return "".join(SUBS[d] for d in str(num))


def varname(i, var):
    return f"{var}{subscript(i)}"

def x_to_bits(x, width):
    bits = [int(b) for b in bin(x)[2:]]
    return [0] * (width - len(bits)) + bits

In [14]:
def get_essential_cover(df):
    columns_to_cover = set(df.columns)
    
    # словарь: строки -> столбцы, которые они покрывают
    row_coverage = {row: set(df.columns[df.loc[row] == 1]) for row in df.index}
    
    selected_rows = set()
    covered_cols = set()

    # 1. Находим существенные строки (которые единственные покрывают какой-то столбец)
    while True:
        essential_found = False
        for col in columns_to_cover - covered_cols:
            covering_rows = [row for row, cols in row_coverage.items() if col in cols]
            if len(covering_rows) == 1:
                row = covering_rows[0]
                selected_rows.add(row)
                covered_cols.update(row_coverage[row])
                row_coverage.pop(row)
                essential_found = True
                break  # пересчёт после удаления
        if not essential_found:
            break

    # 2. Жадно добавляем оставшиеся строки
    while covered_cols != columns_to_cover:
        best_row = max(row_coverage.keys(), key=lambda r: len(row_coverage[r] - covered_cols))
        selected_rows.add(best_row)
        covered_cols.update(row_coverage[best_row])
        row_coverage.pop(best_row)

    conjunct = []

    for row in selected_rows:
        var, value = row.split(" = ")
        
        if value == "0":
            conjunct.append(f"~{var}")
        else:
            conjunct.append(var)
    
    return "".join(conjunct)

In [15]:
S = np.array(
    [0b0000, 0b0001, 0b0010, 0b0100, 0b1000, 0b0011, 0b0111, 0b0101, 0b1001],
    dtype=np.uint8
)

matrix = np.array(
    [
        # S1     S2      S3      S4      S5      S6      S7      S8      S9
        [[3, 2], empty,  empty,  [4, 2], [6, 3], empty,  empty,  empty,  empty ], # S1
        [[2, 3], empty,  empty,  empty,  empty,  [7, 1], empty,  empty,  empty ], # S2
        [[5, 1], empty,  empty,  empty,  empty,  empty,  empty,  empty,  empty ], # S3
        [empty,  empty,  empty,  empty,  empty,  empty,  empty,  [5, 3], empty ], # S4
        [empty,  empty,  empty,  empty,  empty,  empty,  empty,  empty,  [2, 3]], # S5
        [empty,  empty,  [0, 0], empty,  empty,  empty,  [6, 2], empty,  empty ], # S6
        [empty,  empty,  empty,  empty,  empty,  empty,  empty,  [1, 3], empty ], # S7
        [empty,  [2, 3], empty,  empty,  empty,  empty,  empty,  empty,  empty ], # S8
        [empty,  [5, 0], empty,  empty,  empty,  empty,  empty,  empty,  empty ], # S9
    ],
    dtype=np.int32,
)

In [16]:
valid = np.all(~(matrix == empty), axis=2)

pairs = matrix[valid]

max_first = np.max(pairs[:, 0])
max_second = np.max(pairs[:, 1])

first_bits = bit_length(max_first)
second_bits = bit_length(max_second)

function = []

for i, row in enumerate(matrix):
    for col in row:
        if (col == empty).all():
            continue

        x = S[i] * 2**first_bits + col[0]
        y = col[1]

        function.append((x, y))

df = pd.DataFrame(function, columns=["x", "y"])

for i in range(second_bits):
    df[f"y{i}"] = df["y"].apply(lambda v: (v >> i) & 1)

state_bits = bit_length(np.max(S))
bit_width = state_bits + first_bits

for i in range(second_bits):
    ones_x = df[df[f"y{i}"] == 1]["x"].tolist()
    zeros_x = df[df[f"y{i}"] == 0]["x"].tolist()

    print(f"\nТаблицы различия для {varname(i + 1, 'y')}:")

    covers = set()

    for x1 in ones_x:
        bits_x1 = x_to_bits(x1, bit_width)
        table_matrix = []
        col_labels = [bin(x0)[2:].zfill(bit_width) for x0 in zeros_x]

        for j, b1 in enumerate(bits_x1):
            row = []
            for x0 in zeros_x:
                bits_x0 = x_to_bits(x0, bit_width)
                row.append(1 if b1 != bits_x0[j] else 0)
            table_matrix.append(row)

        row_labels = []

        for j in range(bit_width):
            if j < state_bits:
                var = varname(j + 1, "z")
            else:
                var = varname(j + 1 - state_bits, "x")

            row_labels.append(f"{var} = {bits_x1[j]}")

        df_table = pd.DataFrame(table_matrix, index=row_labels, columns=col_labels)
        display(df_table)

        cover = get_essential_cover(df_table)
        print("Покрытие:", cover)

        covers.add(cover)

    print(f"\nФункция выхода для {varname(i + 1, 'y')}:")
    print(" | ".join(covers))


Таблицы различия для y₁:


,0000011,0000100,0011000,0011110,1001101
z₁ = 0,0,0,0,0,1
z₂ = 0,0,0,0,0,0
z₃ = 0,0,0,1,1,0
z₄ = 0,0,0,1,1,1
x₁ = 1,1,0,1,0,0
x₂ = 1,0,1,1,0,1
x₃ = 0,1,0,0,0,1


Покрытие: ~z₃x₂x₁


,0000011,0000100,0011000,0011110,1001101
z₁ = 0,0,0,0,0,1
z₂ = 0,0,0,0,0,0
z₃ = 0,0,0,1,1,0
z₄ = 1,1,1,0,0,0
x₁ = 0,0,1,0,1,1
x₂ = 1,0,1,1,0,1
x₃ = 0,1,0,0,0,1


Покрытие: z₄~z₃~x₁


...

Покрытие: z₃x₂


,0001111,0010101,0011000,1001101
z₁ = 0,0,0,0,1
z₂ = 1,1,1,1,1
z₃ = 1,1,0,0,1
z₄ = 1,0,1,0,0
x₁ = 0,1,1,0,1
x₂ = 0,1,0,0,0
x₃ = 1,0,0,1,0


Покрытие: z₂


,0001111,0010101,0011000,1001101
z₁ = 0,0,0,0,1
z₂ = 1,1,1,1,1
z₃ = 0,0,1,1,0
z₄ = 1,0,1,0,0
x₁ = 0,1,1,0,1
x₂ = 1,0,1,1,1
x₃ = 0,1,1,0,1


Покрытие: z₂

Функция выхода для y₂:
z₃x₂ | ~z₃~z₄ | z₁~z₄ | ~z₃~x₁ | z₂
